In [2]:
import numpy as np
import pandas as pd
import googlemaps
import json
from scipy import ndimage
import matplotlib.pyplot as plt

In [4]:
API_key = 'API Key'
gmaps = googlemaps.Client(key=API_key)

In [29]:
geometryDict = {
  "Point": 0,
  "LineString": 1,
  "MultiLineString": 2,
  "Polygon": 2,
  "MultiPolygon": 3
}
buildings = {'yes','house','university','dormitory','apartments','detached',
             'hotel','residential','commercial','office','retail',
             'supermarket','church','hospital','government','school','college'}

warehouses = {'way/174188355', 'way/676434395', 'way/440108548', 'way/447802001', 'way/453179452', 
              'way/38263351', 'way/173503696', 'way/37749300', 'way/412588608', 'way/958528168',
              'way/38953076', 'way/39015619', 'way/422307109', 'way/102878296'}

f = open('uw.geojson')
load = json.load(f)
columnNames = ["ID", "building type", "latitudes", "longitudes"]
df = pd.DataFrame(columns = columnNames)
wf = pd.DataFrame(columns = columnNames)
i = 0
j = 0
for feature in load['features']:
  if 'building' in feature['properties'] and feature['properties']['building'] in buildings:
    geometry = feature['geometry']
    id = feature['properties']['@id']
    type = geometry['type']
    coords = geometry['coordinates']
    buildingType = feature['properties']['building']
    if id in warehouses:
      if geometryDict[type] == 2:
        wf.loc[i] = [id] + [buildingType] + [coords[0][0][1]] + [coords[0][0][0]]
      elif geometryDict[type] == 1:
        wf.loc[i] = [id] + [buildingType] + [coords[0][1]] + [coords[0][0]]
      elif geometryDict[type] == 0:
        wf.loc[i] = [id] + [buildingType] + [coords[1]] + [coords[0]]
      i = i+1
    else:
      if geometryDict[type] == 2:
        df.loc[j] = [id] + [buildingType] + [coords[0][0][1]] + [coords[0][0][0]]
      elif geometryDict[type] == 1:
        df.loc[j] = [id] + [buildingType] + [coords[0][1]] + [coords[0][0]]
      elif geometryDict[type] == 0:
        df.loc[j] = [id] + [buildingType] + [coords[1]] + [coords[0]]
      j = j+1

In [30]:
destinations = list(zip(df.latitudes, df.longitudes))
ware_destinations = list(zip(wf.latitudes, wf.longitudes))

In [31]:
durations = []

for ware_destination in ware_destinations:
    actual_duration = []
    for destination in destinations:
        result = gmaps.distance_matrix(ware_destination, destination, mode='driving')["rows"][0]["elements"][0]["duration"]["value"]
        actual_duration.append(result)
    durations.append(actual_duration)

distances = pd.DataFrame(durations).T

In [32]:
df.to_csv('buildings.csv', sep='\t')
wf.to_csv('warehouses.csv', sep='\t')
distances.to_csv('distances.csv', sep='\t')